# Informed search - the A* algorithm

The depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [2]:
class Problem:
    @property
    def initial_state(self):
        0
        
    def available_actions(self, state):
        0    
        
    def take_action(self, state, action):
        0
        # return new_state
    
    def is_goal(self, state) -> bool:
        0
        
    def action_cost(self, state, action) -> float:
        0
        
    def heuristic(self, state) -> float:
        0

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [3]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def take_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a **priority queue**. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
   1. Pick an item with the lowest expected cost
   2. If this is the goal, return the sequence of actions necessary to reach this state
   3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.

Keep track of visited states - it may be necessary to discover that the problem has no solutions. Assume states are hashable.
  
In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search (i.e., the number of times you removed an element from the queue) and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [12]:
import heapq

def astar(problem: Problem):
    initial_state = problem.initial_state
    queue = [(problem.heuristic(initial_state), initial_state, [], 0)]
    # expected_costs = {initial_state: problem.heuristic(initial_state)}
    best_costs = {initial_state: 0}
    counter = 0

    while queue:
        total_cost, cur_state, cur_actions, cur_cost = heapq.heappop(queue)
        counter += 1
        if problem.is_goal(cur_state):
            print(counter) # wait for goal-checking until it's the cheapest way
            return cur_actions
        for action in problem.available_actions(cur_state):
            new_state = problem.take_action(cur_state, action)
            new_cost = cur_cost + problem.action_cost(cur_state, action)
            new_total_cost = new_cost + problem.heuristic(new_state)
            new_actions = cur_actions + [action]

            if new_cost >= best_costs.get(new_state, float("inf")):
                continue # already a better path found to this node
            # if problem.is_goal(new_state):
            #     print(counter)
            #     return new_actions
            best_costs[new_state] = new_cost # update whenever better solution found
            heapq.heappush(queue, (new_total_cost, new_state, new_actions, new_cost))
    print(counter)
    return None

        

Now lets test your code in the vacuum world!

In [13]:
print(astar(VacuumProblem()))

4
['Suck', 'Right', 'Suck']


## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [14]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

7


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [15]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

4


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

VacuumProblem and VacuumProblem2 noth work best - the second one is not expected given the properties of the heuristics. It may be that this approach finds the solution the fastest because overestimated costs of heuristic encourage to explore more "depth" solutions as cost plays a lesser role here (being outnumbered by heuristic) - it's enough to clean one room to "earn" 9 additional moves. Because of that it explores less solutions but "checks them out" in details more quickly - thanks to what it's so fast on little datasets/examples but would struggle in more complex environments

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time **don't change the initial state (nor the goal state)**, your solution should be capable enough to solve this. The classes are named `PuzzleProblem1`, `PuzzleProblem2`, `PuzzleProblem3` - remember not to rename them!

![](aima-images/fig3_4.png)

In [16]:
class PuzzleProblem1(Problem):
    @property
    def initial_state(self):
        return (4, (7,2,4,5,0,6,8,3,1))
        
    def available_actions(self, state):
        return ["Up", "Right", "Down", "Left"]
        
    def take_action(self, state, action):
        robot, dirty = state

        new_row = robot // 3
        new_column = robot % 3
        if action == "Up":
            new_row = max(new_row-1, 0)
        elif action == "Right":
            new_column = min(new_column+1, 2)
        elif action == "Down":
            new_row = min(new_row+1, 2)
        elif action == "Left":
            new_column = max(new_column-1, 0)
        else:
            raise Exception("Invalid action")
        
        new_dirty = list(dirty)
        new_robot = new_row * 3 + new_column
        new_dirty[new_robot], new_dirty[robot] = new_dirty[robot], new_dirty[new_robot]

        return (new_robot, tuple(new_dirty))
    
    def is_goal(self, state) -> bool:
        return state[1] == tuple(range(9))
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        # dont cound a blank space
        wrong_positions = (state[1][i] != 0 and state[1][i] != i for i in range(9))
        return sum(wrong_positions) # counting wrong positions -> the closer to the goal the lower the heuristic

**Prove that this heuristic is admissible.**
With each move we can lower our heuristic expectation by at most 1 (thanks to the fact that the blank space is excluded from computing the heuristic value - otherwise if you swapped for instance 1 and 0 to their correct positions - you'd "move" closer to the solution by 2 while "paying" 1 for the action), and each action costs 1 so we can never "outrun" the heuristic estimation with even the most efficient actions - we can at most be as good as them - which proves that we never overestimate heuristic values

*Replace this text with an answer*

In [17]:
class PuzzleProblem2(PuzzleProblem1):
    def heuristic(self, state) -> float:     
        total_distance = 0
        for cur_index in range(9):
            cur_node = state[1][cur_index]
            if cur_node == 0:
                continue
            cur_row = cur_index // 3
            cur_column = cur_index % 3
            target_row = cur_node // 3
            target_column = cur_node % 3
            total_distance += abs(cur_row - target_row) + abs(cur_column - target_column)
        return total_distance

**Prove that this heuristic is admissible.**
With each move we come "closer" to the solution by at most 1 (we cannot move diagonally and we don't take a blank space into account - so with each move just one node changes its position), each move also costs 1 so we never "outrun" heuristic estimation even with the most efficient moves possible - which proves we never overestimate heuristic values

*Replace this text with an answer*

In [18]:
class PuzzleProblem3(PuzzleProblem1):
    def heuristic(self, state) -> float:
        euclidean_distance = 0
        for cur_index in range(9):
            cur_node = state[1][cur_index]
            if cur_node == 0:
                continue
            cur_row = cur_index // 3
            cur_column = cur_index % 3
            target_row = cur_node // 3
            target_column = cur_node % 3
            euclidean_distance += ((cur_row-target_row)**2+(cur_column-target_column)**2)**0.5
        return euclidean_distance


**Prove that this heuristic is admissible.**

*Replace this text with an answer*
With each move we can lower the heuristic estimation by at most 1 - when we are in the same row or column as the target place and move towards it (we ignore the blank space so with each move just one node is "affected" in the heuristic function), and each action costs 1 so we never "outrun" our estimations. Also - euclidean distance is kind of like an diagonal distance which never overestimates our path as we can move only up, down, left and right (not diagonally)

Run your heuristics on the given problem. **All three plans should be of the same length**, but they may differ.

In [19]:
plan0 = astar(PuzzleProblem1())
plan1 = astar(PuzzleProblem2())
plan2 = astar(PuzzleProblem3())

print("Is plan0==plan1?", plan0 == plan1)
print("Is len(plan0)==len(plan1)?", len(plan0) == len(plan1))
print("Is plan0==plan2?", plan0 == plan2)
print("Is len(plan0)==len(plan2)?", len(plan0) == len(plan2))
print("Is plan1==plan2?", plan1 == plan2)
print("Is len(plan1)==len(plan2)?", len(plan1) == len(plan2))

38887
2521
6638
Is plan0==plan1? True
Is len(plan0)==len(plan1)? True
Is plan0==plan2? True
Is len(plan0)==len(plan2)? True
Is plan1==plan2? True
Is len(plan1)==len(plan2)? True


**Which of the heuristics is the best for this task? Why is that?**

The best heuristics for this task is the second one - Manhattan distance. It is because of the fact that it is the most informative one - the first one provides just how many nodes are on wrong positions - and leaves no hints on how big those "misplacements" are while Euclidean distance in the third approach gives us smaller values than Manhattan distance making it worse at performing estimations (underestimates more than Manhattan distance)

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.